# Logistic regression and comparing groups

Idea: Could we any of:
- the Titanic dataset
- the credit scoring dataset (from the lectures)
- the dataset to with depression and body image (from the coursework)

in two ways:

1. Compare differences in means between two groups (hypothesis test)
   - Using bootstrap
   - Using t-test

2. Try to fit a logistic regression model to predict the survival (in Titanic) or credit approval 

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


In [ ]:
titanic = pd.read_csv('datasets/titanic.csv')
sns.distplot(titanic.loc[titanic['Survived'] == 1, 'Age'])

In [ ]:
sns.pairplot(titanic, hue='Survived')

In [ ]:
titanic[titanic['Fare'] == 0]

## Transform titanic suitable for sk-learn
- Replace the `Fare` variable with a log transformed version of `Fare` called `LogFare`. What's the problem with a log transform? Hint: you can fix this by the transform `log(x) + 1`
- Encode female/male in 'Sex' as 0/1 values
- Drop any other non-numeric variables

In [ ]:
titanic_trans = titanic.copy()

In [ ]:
titanic_trans['LogFare'] = np.log10(titanic['Fare'] + 1)
titanic_trans.drop(['Fare', 'Name'], 1, inplace=True)
titanic_trans.replace({'Sex': {'female': 0, 'male': 1}}, inplace=True)

In [ ]:
sns.pairplot(titanic_trans, hue='Survived')

In [ ]:
titanic_sample = titanic.sample(100, random_state=6)
# sns.relplot(x='Age', y='Survived', data=titanic_sample)

In [ ]:
X = titanic_trans.drop('Survived', 1).to_numpy(copy=True) 
y = titanic_trans['Survived'].to_numpy()
clf = LogisticRegression(random_state=0).fit(X, y)
clf.intercept_
clf.coef_

**Exercise:** Interpret the intercept, including the characteristics of the passenger to which it applies. Does such a passenger exist?

In [ ]:
clf.intercept_

This is the log odds of survival of a newborn female in "zeroeth class" with no siblings and no parents aboard, who paid no fare. It shows that this hypothetical passenger was $e^3.701=40.5$ times more likely to survive than die.

**Exercise:** Interpret the coefficients. You may find it helpful to convert the output from sklearn back into a pandas Series with an index. Try to use language that you think would be understandable by a general audience. [Paste it into the form here: we will have a poll to decide who made the most understandable interpretation.]

In [ ]:
coeffs = pd.Series(clf.coef_[0], index=titanic_trans.columns.drop('Survived'))

In [ ]:
coeffs

In [ ]:
np.exp(coeffs)

All other things being equal:
- For each class higher, passengers were about 2.6 times (1/0.386) more likely to survive than not - i.e. 1st class passengers were 2.6 times as likely to survive as second class passengers
- Men were 13.6 times less likely to survive than women
- Every year of age meant that your odds of survival to drowning went down by about 4%
- For every sibling or spouse aboard, your odds of survival decreased by a factor of 1.6
- For every sibling or spouse aboard, your odds of survival decreased by a factor of 1.82
- A passenger who paid £100 was 2.5 times as likely to survive as one who paid £10.  

## How many of these coefficients are meaningful?

How likely is it that some of these coefficients could have arisen by chance? We'd like to find confidence intervals for each coefficient. 

**Excercise:** Write a bootstrap function to generate the sampling distribution of all of the coefficients. On each bootstrap iteration, we'd like to store the values of the intercept and all of the coefficients in one row of a dataframe. We'll then be able to plot distribution of the dataframe, and compute confidence intervals from the marginal distributions.

In [ ]:
def titanic_lr(titanic_trans):
    X = titanic_trans.drop('Survived', 1).to_numpy(copy=True) 
    y = titanic_trans['Survived'].to_numpy()
    # pipe = make_pipeline(StandardScaler(), LogisticRegression())
    clf = LogisticRegression().fit(X, y)
    coeffs = pd.Series(clf.intercept_, index=['Intercept'])
    coeffs = coeffs.append(pd.Series(clf.coef_[0], index=titanic_trans.columns.drop('Survived')))
    return(coeffs)

In [ ]:
titanic_lr(titanic_trans)

In [ ]:
def bootstrap_df(df, k=1000, estimator=titanic_lr, quantiles=[0.025, 0.975], plot=False):
    n = len(df)
    coeffs = estimator(df)
    x_star_est = pd.DataFrame(index=range(k), columns=coeffs.index)
    for i in range(k):
        x_star = df.sample(n, replace=True)
        x_star_est.loc[i] = estimator(x_star)
        
    return(x_star_est, x_star_est.quantile(quantiles), x_star_est.std())

In [ ]:
titanic_lr_bs = bootstrap_df(titanic_trans)

In [ ]:
sns.pairplot(titanic_lr_bs[0])